In [36]:
import sys
print(sys.executable)
!{sys.executable} -m pip install gurobipy
from gurobipy import *

/Users/Abe/opt/anaconda3/bin/python


In [37]:
import math

In [44]:
# create a new model
myModel = Model( "ClientDepot" )

n = 3
n_min = 2
n_max = 10
d_max = 10

# pretend X (only works with n = 3)!**
X = [[30, 40, 20],
     [10, 60, 20]]

In [45]:
# create decision variables and integrate them into the model

Y = []
for i in range(n):
    Y.append([])
    for j in range(n):
        Y[i].append(0)
        Y[i][j] = myModel.addVar(lb = 0, ub = 1.0, vtype = GRB.BINARY )

# Y is an assignment matrix:
# Y[i][j] = 1: client i is assigned to depot j, otherwise 0
# each entry of Y is its own decision variable
# print(Y)


depots = [myModel.addVar(lb = 0, ub = 1.0, vtype = GRB.BINARY ) for j in range(n)]
#depots is a vector that indicates whether a depot has clients assigned to it
#depots[j] = 0: no clients assigned to depot j.
#          = 1: one or more clients assigned to depot j.

clients = [None for depot in range(n)] 
#parallel (perpendicular?) list to depots, used just for summing 


#Y = myModel.addVar( vtype = GRB.BINARY, name = "Y" )
myModel.update()
print(Y)
        

[[<gurobi.Var C0>, <gurobi.Var C1>, <gurobi.Var C2>], [<gurobi.Var C3>, <gurobi.Var C4>, <gurobi.Var C5>], [<gurobi.Var C6>, <gurobi.Var C7>, <gurobi.Var C8>]]


In [46]:
# create a linear expression for the objective

objExpr = LinExpr()

for j in range(len(depots)):
    objExpr += depots[j]

myModel.setObjective( objExpr , GRB.MINIMIZE )

print(objExpr)

<gurobi.LinExpr: C9 + C10 + C11>


In [47]:
# create expressions for constraints and add to the model

#definition of depots (lil y) constraint
#logic:

#if there are some clients assigned to depot j (nonzero entries in the j-th col of Y)
#then depots[j] must be non-zero. since depots is binary, it will take on the value 1.

#if there are no clients assigned to depot j (j-th col of Y is all zeros),
#then depots[j] will be zero since our objective minimizes the sum of the entries of depots

firstConst = [LinExpr() for j in depots]
for j in range(len(depots)):
    
    for i in range(len(clients)):
        firstConst[j] += Y[i][j]/(n+1)
        
    myModel.addConstr( lhs = firstConst[j] , sense = GRB.LESS_EQUAL , rhs = depots[j] )
    
    
#client-depot uniqueness constraint:

#ensures that the sum of each row of Y is exactly 1 
#(each client assigned to one unique depot)

secondConst = [LinExpr() for i in clients]
for i in range(len(clients)):
    
    for j in range(len(depots)):
        secondConst[i] += Y[i][j]
    
    myModel.addConstr( lhs = secondConst[i] , sense = GRB.EQUAL , rhs = 1 )

    
#min and max clients constraint:

thirdConst = [LinExpr() for j in depots]
for j in range(len(depots)):
    
    for i in range(len(clients)):
        thirdConst[j] += Y[i][j]
    
    myModel.addConstr( lhs = thirdConst[j] , sense = GRB.LESS_EQUAL , rhs = n_max*depots[j] )
    myModel.addConstr( lhs = thirdConst[j] , sense = GRB.GREATER_EQUAL , rhs = n_min*depots[j] )
    
#max distance constraint:

#fourthConst = [LinExpr() for j in depots]
for j in range(len(depots)):
    
    #compare the distances between all clients in the column
    
    for i in range(len(depots)):
        for ii in range(len(depots)):
            
            d = math.sqrt((X[0][i] - X[0][ii])**2 + (X[1][i] - X[1][ii])**2)
            act = 10000*(1 - Y[i][j]) + 10000*(1 - Y[ii][j])
            #if Y[i][j] or Y[ii][j] = 0, the constraint will be satisfied automatically
            
            myModel.addConstr( lhs = d , sense = GRB.LESS_EQUAL , rhs = d_max + act )
            
    



    
    
    
    
# firstConst += 100 * xm
# firstConst += 40 * xs
# myModel.addConstr( lhs = firstConst , sense = GRB.LESS_EQUAL , \
# rhs = 1 , name = "client" )


# secondConst = LinExpr()
# secondConst += 200 * xm
# secondConst += 400 * xs
# myModel.addConstr( lhs = secondConst , sense = GRB.LESS_EQUAL , \
# rhs = 60000 , name = "stoConst" )

In [48]:
# integrate objective and constraints into the model
myModel.update()
# write the model in a file to make sure it is constructed correctly
myModel.write( filename = "testOutput.lp" )
# optimize the model
myModel.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 39 rows, 12 columns and 90 nonzeros
Model fingerprint: 0x145d0327
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+04]
Presolve removed 30 rows and 12 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [49]:
# print optimal objective and optimal solution
print( "\nOptimal Objective: " + str( myModel.ObjVal ) )
print( "\nOptimal Solution:" )
myVars = myModel.getVars()
for curVar in myVars:
    print ( curVar.varName + " " + str( curVar.x ) )
# print optimal dual solution
print( "\nOptimal Dual Solution:" )
myConsts = myModel.getConstrs()
for curConst in myConsts:
    print ( curConst.constrName + " " + str( curConst.pi ) )

AttributeError: Unable to retrieve attribute 'ObjVal'